### Importing Libraries



In [0]:
import requests
from pyspark.sql.functions import col

###Defining Variables 

In [0]:
NEWSAPI_KEY = "695dc9ebf94a448fa3a078a811564f6f"

### Connecting to API and checking response 

In [0]:
api_url = (
    "https://newsapi.org/v2/top-headlines"
    "?country=us"
    f"&apiKey={NEWSAPI_KEY}"
)
response = requests.get(api_url)
news_json = response.json()

if response.status_code != 200 or "articles" not in news_json:
    raise Exception(
        f"API request failed: {news_json.get('message', 'Unknown error')}"
    )


### Creating Pyspark DataFrame

In [0]:
raw_df = spark.createDataFrame(news_json["articles"])

raw_df.printSchema()

### Displaying DataFrame

In [0]:
display(raw_df)

### Transformation 

In [0]:



news_df = raw_df.select(
    col("source.name").alias("source_name"),
    col("author"),
    col("title"),
    col("description"),
    col("url"),
    col("publishedAt").alias("published_at"),
    col("content")
)



### Displaying Transformation

In [0]:
display(news_df)

### Record that contains any news related to Russia or about Trump or the article written by Rebecca Falconer

In [0]:
news_df = spark.read.table(
    'workspace.news_db.top_headlines'
)

filtered_df = news_df.filter(
    (news_df.title.contains('Russia')) |
    (news_df.title.contains('Trump')) |
    (news_df.author == 'Rebecca Falconer')
)


### Filtered Table

In [0]:
display(filtered_df)

In [0]:
news_df.write.mode("append").saveAsTable("news_db.top_headlines")